# This file is preprocessing of the annotated data

code reference for preprocessing taken from https://github.com/walidamamou/layoutlmV2/blob/0f1bc7df6aa875f6fe68f8daae87dd38781685be/preprocess.py

i did the manual annotation using uibai annotation tool

In [ ]:
!pip install transformers
!pip install -q datasets seqeval
!pip install pyyaml==5.1
!pip install torch==1.10.0+cu111 torchvision==0.11.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html
!python -m pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.10/index.html

In [2]:
import os
os.chdir('/content/drive/MyDrive/unstop_competition/')

In [3]:
import pandas as pd
import numpy as np
import os
import argparse

from PIL import Image
from datasets import Features, Sequence, ClassLabel, Value, Array2D, Array3D,Dataset
from transformers import LayoutLMv2Processor
from sklearn.model_selection import train_test_split


import warnings
warnings.filterwarnings('ignore')

def read_text_file(file_path):
    with open(file_path, 'r') as f:
        return (f.readlines())

def get_zip_dir_name():
  zip_dir_name = "train_kxcu7ux.zip"
  return zip_dir_name

def preprocess_data(examples):
  images = [Image.open(path).convert("RGB") for path in examples['image_path']]
  words = examples['words']
  boxes = examples['bboxes']
  word_labels = examples['ner_tags']
  
  encoded_inputs = processor(images,words, boxes=boxes, word_labels=word_labels,
                             padding="max_length", truncation=True)
  
  return encoded_inputs




In [4]:
files = {}
zip_dir_name = get_zip_dir_name()
if zip_dir_name:
  files['train_box'] = read_text_file(os.path.join("data","train_kxcu7ux","fd5512e3-caf2-4ea1-8816-816d9bc02dab","fd5512e3-caf2-4ea1-8816-816d9bc02dab_box.txt"))
  files['train_image'] = read_text_file(os.path.join("data","train_kxcu7ux","fd5512e3-caf2-4ea1-8816-816d9bc02dab","fd5512e3-caf2-4ea1-8816-816d9bc02dab_image.txt"))
  files['train'] = read_text_file(os.path.join("data","train_kxcu7ux","fd5512e3-caf2-4ea1-8816-816d9bc02dab","fd5512e3-caf2-4ea1-8816-816d9bc02dab.txt"))

In [5]:
assert(len(files['train']) == len(files['train_box']))
assert(len(files['train_box']) == len(files['train_image']))
assert(len(files['train_image']) == len(files['train']))


In [6]:
images = {}
for i,row in enumerate(files['train_image']):
  if row != '\n':
    image_name = row.split('\t')[-1]
    images.setdefault(image_name.replace('\n',''),[]).append(i)

In [7]:
words,bboxes,ner_tags,image_path= [],[],[],[]
for image,rows in images.items():
    words.append([row.split('\t')[0].replace('\n','') for row in files['train'][rows[0]:rows[-1]+1]])
    ner_tags.append([row.split('\t')[1].replace('\n','') for row in files['train'][rows[0]:rows[-1]+1]])
    bboxes.append([box.split('\t')[1].replace('\n','') for box in files['train_box'][rows[0]:rows[-1]+1]])
    if zip_dir_name:
        image_path.append(f"data/train_kxcu7ux/fd5512e3-caf2-4ea1-8816-816d9bc02dab/{image}")
    else:
        image_path.append(f"data/train_kxcu7ux/fd5512e3-caf2-4ea1-8816-816d9bc02dab/{image}")

In [8]:
labels = list(set([tag for doc_tag in ner_tags for tag in doc_tag]))  
id2label = {v: k for v, k in enumerate(labels)}
label2id = {k: v for v, k in enumerate(labels)}


In [9]:
class data_config:
        labels = np.unique([tag for doc_tag in ner_tags for tag in doc_tag]).tolist()
        num_labels = len(labels)
        id2label = {v: k for v, k in enumerate(labels)}
        label2id = {k: v for v, k in enumerate(labels)}

dataset_dict = {
        'id' : range(len(words)),
        'words' : words,
        'bboxes' : [[list(map(int,bbox.split())) for bbox in doc]  for doc in bboxes],
        'ner_tags' : [[data_config.label2id[tag] for tag in ner_tag] for ner_tag in ner_tags],
        'image_path' : image_path
    }

df = pd.DataFrame(dataset_dict)
df.id = df.id.apply(str)
assert df.shape[0] == len(images)


In [10]:
train = df

In [11]:
#filter out untagged images to prevent 'O' tag overfitting (UNDERSAMPLINIG)
limit = 25 # we do not accept to reduce the train dataset with more than 25% of it's original size 
untagged_images = pd.DataFrame(columns=train.columns)
filtered_train = train.copy()
for index,row in filtered_train.iterrows():
    tags = row['ner_tags']
    if all([tag == data_config.label2id['O'] for tag in tags]):
      untagged_images = untagged_images.append(row)
      filtered_train = filtered_train.drop(index = index)
      if filtered_train.shape[0] <= ((100-limit)/100) * train.shape[0]:
        break
features = Features({
    'id': Value(dtype='string', id=None),
    'words': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
    'bboxes' : Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
    'ner_tags' : Sequence(feature=ClassLabel(num_classes=len(data_config.labels),names=data_config.labels, names_file=None, id=None), length=-1, id=None),
    'image_path' : Value(dtype='string', id=None)
})

In [12]:
train_dataset = Dataset.from_pandas(filtered_train,features=features)

In [14]:
processor = LayoutLMv2Processor.from_pretrained("microsoft/layoutlmv2-base-uncased", revision="no_ocr")

In [30]:
train_dataset.map

{'id': Value(dtype='string', id=None),
 'words': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'bboxes': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['B-BILL_TO_ADDRESS', 'B-BILL_TO_NAME', 'B-DUE_DATE', 'B-INVOICE_DATE', 'B-INVOICE_NUMBER', 'B-PAYMENT_INFO', 'B-SHIP_TO_ADDRESS', 'B-SHIP_TO_NAME', 'B-TOTAL_AMOUNT', 'E-BILL_TO_ADDRESS', 'E-BILL_TO_NAME', 'E-DUE_DATE', 'E-INVOICE_DATE', 'E-INVOICE_NUMBER', 'E-PAYMENT_INFO', 'E-SHIP_TO_ADDRESS', 'E-SHIP_TO_NAME', 'E-TOTAL_AMOUNT', 'I-BILL_TO_ADDRESS', 'I-BILL_TO_NAME', 'I-DUE_DATE', 'I-INVOICE_DATE', 'I-PAYMENT_INFO', 'I-SHIP_TO_ADDRESS', 'I-SHIP_TO_NAME', 'I-TOTAL_AMOUNT', 'O', 'S-BILL_TO_ADDRESS', 'S-BILL_TO_NAME', 'S-DUE_DATE', 'S-INVOICE_DATE', 'S-INVOICE_NUMBER', 'S-PAYMENT_INFO', 'S-SHIP_TO_ADDRESS', 'S-TOTAL_AMOUNT', 'S-TOTAL_TAX_AMOUNT'], id=None), length=-1, id=None),
 'image_path': Value(dtype='st

In [16]:
# we need to define custom features
features = Features({
    'image': Array3D(dtype="int64", shape=(3, 224, 224)),
    'input_ids': Sequence(feature=Value(dtype='int64')),
    'attention_mask': Sequence(Value(dtype='int64')),
    'token_type_ids': Sequence(Value(dtype='int64')),
    'bbox': Array2D(dtype="int64", shape=(512, 4)),
    'labels': Sequence(ClassLabel(names=data_config.labels)),
}
)

In [17]:
train_dataset = train_dataset.map(preprocess_data, batched=True, remove_columns=train_dataset.column_names,features=features)
train_dataset.set_format(type="torch", device="cuda")

Map:   0%|          | 0/25 [00:00<?, ? examples/s]

In [21]:
OUTPUT_PATH = "data/preprocessed_data_train/"

In [22]:
train_dataset.save_to_disk(f'{OUTPUT_PATH}')

Saving the dataset (0/1 shards):   0%|          | 0/25 [00:00<?, ? examples/s]